In [1]:
import numpy as np
import pandas as pd
from collections import OrderedDict

import praw

import json

from datetime import datetime

pd.options.display.max_colwidth = 75

In [2]:
df = pd.read_csv('../data/movies_cleaned.csv', index_col='id')

In [3]:
with open('../.secret/ZSDSFI_client_id.txt') as f:
    client_id = f.read()

with open('../.secret/ZSDSFI_client_secret.txt') as f:
    client_secret = f.read()

In [4]:
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent="Movie Scraper by ZSDSFI"
)

In [5]:
df.head()

,title,post_title,url
id,,,
vzcwal,the princess,official discussion - the princess [spoilers],https://www.reddit.com/r/movies/comments/vzcwal/official_discussion_the...
vzcw0a,the man from toronto,official discussion - the man from toronto [spoilers],https://www.reddit.com/r/movies/comments/vzcw0a/official_discussion_the...
vzcvsd,the sea beast,official discussion - the sea beast [spoilers],https://www.reddit.com/r/movies/comments/vzcvsd/official_discussion_the...
vzcvkz,mrs. harris goes to paris,official discussion - mrs. harris goes to paris [spoilers],https://www.reddit.com/r/movies/comments/vzcvkz/official_discussion_mrs...
vzcv66,where the crawdads sing,official discussion - where the crawdads sing [spoilers],https://www.reddit.com/r/movies/comments/vzcv66/official_discussion_whe...


In [6]:
all_ids = df.index.tolist()

In [7]:
# The following cell can about one hour to run.

In [8]:
movies_comments = {}

for id in all_ids:
    submission = reddit.submission(id=id)
    comments = []
    # Using a try-except block because at least one movie causes an error, as the thread no longer exists 
    try:
        submission.comment_sort = 'top'
        post_date_utc = submission.created_utc
        for top_level_comment in submission.comments[:100]:
            comments.append(top_level_comment.body)
    except:
        continue
    movies_comments[id] = {
        'title': df.loc[id]['title'],
        'comments': comments,
        'post_date_utc': post_date_utc
    }

In [9]:
pd.DataFrame(movies_comments).T.info()

<class 'pandas.core.frame.DataFrame'>
Index: 944 entries, vzcwal to 47szbr
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          944 non-null    object
 1   comments       944 non-null    object
 2   post_date_utc  944 non-null    object
dtypes: object(3)
memory usage: 29.5+ KB


In [10]:
pd.DataFrame(movies_comments).T.sample(10)

,title,comments,post_date_utc
d6o3sm,downton abbey,[Congrats to John and Anna Bates for both managing to stay out of priso...,1568945046.0
lj1cfb,judas and the black messiah,[The ‘I AM A REVOLUTIONARY’ scene inside the church is absolutely pheno...,1613225944.0
dgpkzq,mister america,[Finally. They told me I was a fool. To waste my time watching fake mov...,1570849758.0
oqy06h,blood red sky,[It was surprisingly better than I had hoped for. A solid horror film a...,1627160050.0
6jxijs,baby driver,[I don't think I've ever seen so many different things timed to music b...,1498614999.0
p8fnnq,annette,"[""Annette"" is 100% a love it or hate it film. But you need to admit tha...",1629499166.0
5697im,queen of katwe,[I enjoyed it so much more than I expected too. Even though there wasn'...,1475807469.0
dzlep9,a beautiful day in the neighborhood,[Absolutely loved this film. Loved the use of miniatures for scene chan...,1574391631.0
pprhvt,dune,"[Seeing Jason Momoa clean shaven shook me to my core.\n\nVisually, the ...",1631845730.0
qx6sxk,ghostbusters: afterlife,[Easily the most unrealistic part of the movie was how pristine that Wa...,1637291428.0


In [11]:
# Saving the lists of movie comments as a JSON file
# Run this once

# with open("../data/movies_comments.json", "w") as outfile:
#     json.dump(movies_comments, outfile)